In [1]:
# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from datetime import datetime as dt

# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline


In [3]:
import xml.etree.ElementTree as ET
from collections import namedtuple

import requests

MUNI_METRO_ROUTES = frozenset(["N", "M", "L", "KT", "J"])
Vehicle = namedtuple("Vehicle", "lat lon route_tag")

def parse_muni_data(api_response):

    vehicles = ET.fromstring(api_response).getchildren()
    vehicles = [
        v for v in vehicles
        if v.attrib.get('routeTag') in MUNI_METRO_ROUTES
    ]
    filtered_vehicles = [
        Vehicle(v.attrib['lat'], v.attrib['lon'], v.attrib['routeTag'])
        for v in vehicles
        if 'lat' in v.attrib and 'lon' in v.attrib
    ]
    return filtered_vehicles

def get_raw_muni_data():
    return requests.get("http://webservices.nextbus.com/service/publicXMLFeed?command=vehicleLocations&a=sf-muni&t=0").text

locs = parse_muni_data(get_raw_muni_data())


In [4]:
locs

[Vehicle(lat='37.7432', lon='-122.47263', route_tag='L'),
 Vehicle(lat='37.71237', lon='-122.40177', route_tag='KT'),
 Vehicle(lat='37.7930599', lon='-122.3965', route_tag='N'),
 Vehicle(lat='37.77726', lon='-122.41643', route_tag='L'),
 Vehicle(lat='37.7237099', lon='-122.44414', route_tag='J'),
 Vehicle(lat='37.76526', lon='-122.38898', route_tag='KT'),
 Vehicle(lat='37.76183', lon='-122.47431', route_tag='N'),
 Vehicle(lat='37.74239', lon='-122.4906', route_tag='L'),
 Vehicle(lat='37.77565', lon='-122.39498', route_tag='N'),
 Vehicle(lat='37.73614', lon='-122.50467', route_tag='L'),
 Vehicle(lat='37.76914', lon='-122.38932', route_tag='KT'),
 Vehicle(lat='37.75869', lon='-122.44137', route_tag='L'),
 Vehicle(lat='37.7142399', lon='-122.4683', route_tag='M'),
 Vehicle(lat='37.76598', lon='-122.4289', route_tag='J'),
 Vehicle(lat='37.74855', lon='-122.38744', route_tag='N'),
 Vehicle(lat='37.7746', lon='-122.41979', route_tag='N'),
 Vehicle(lat='37.74203', lon='-122.46502', route_tag=